### location file format : map-making-app -> GeoGuessr

In [1]:
import json
from geopy.geocoders import Nominatim

In [22]:
country_dict = {"Aruba": "アルバ", "Afghanistan": "アフガニスタン", "Angola": "アンゴラ", "Anguilla": "アンギラ", "Åland Islands": "オーランド諸島", "Albania": "アルバニア", "Andorra": "アンドラ", "United Arab Emirates": "アラブ首長国連邦", "Argentina": "アルゼンチン", "Armenia": "アルメニア", "American Samoa": "アメリカ領サモア", "Antarctica": "南極大陸", "French Southern Territories": "フランス領南方地域", "Antigua and Barbuda": "アンティグア・バーブーダ", "Australia": "オーストラリア", "Austria": "オーストリア", "Azerbaijan": "アゼルバイジャン", "Burundi": "ブルンジ", "Belgium": "ベルギー", "Benin": "ベナン", "Bonaire, Sint Eustatius and Saba": "ボネール、シント・ユースタティウスおよびサバ", "Burkina Faso": "ブルキナファソ", "Bangladesh": "バングラデシュ", "Bulgaria": "ブルガリア", "Bahrain": "バーレーン", "Bahamas": "バハマ", "Bosnia and Herzegovina": "ボスニア・ヘルツェゴビナ", "Saint Barthélemy": "サン・バルテルミー", "Belarus": "ベラルーシ", "Belize": "ベリーズ", "Bermuda": "バミューダ", "Bolivia, Plurinational State of": "ボリビア、多民族国", "Brazil": "ブラジル", "Barbados": "バルバドス", "Brunei Darussalam": "ブルネイ・ダルサラーム", "Bhutan": "ブータン", "Bouvet Island": "ブーベ島", "Botswana": "ボツワナ", "Central African Republic": "中央アフリカ共和国", "Canada": "カナダ", "Cocos (Keeling) Islands": "ココス（キーリング）諸島", "Switzerland": "スイス", "Chile": "チリ", "China": "中国", "Côte d'Ivoire": "コートジボワール", "Cameroon": "カメルーン", "Congo, The Democratic Republic of the": "コンゴ民主共和国", "Congo": "コンゴ", "Cook Islands": "クック諸島", "Colombia": "コロンビア", "Comoros": "コモロ", "Cabo Verde": "カーボベルデ", "Costa Rica": "コスタリカ", "Cuba": "キューバ", "Curaçao": "キュラソー", "Christmas Island": "クリスマス島", "Cayman Islands": "ケイマン諸島", "Cyprus": "キプロス", "Czechia": "チェコ", "Germany": "ドイツ", "Djibouti": "ジブチ", "Dominica": "ドミニカ国", "Denmark": "デンマーク", "Dominican Republic": "ドミニカ共和国", "Algeria": "アルジェリア", "Ecuador": "エクアドル", "Egypt": "エジプト", "Eritrea": "エリトリア", "Western Sahara": "西サハラ", "Spain": "スペイン", "Estonia": "エストニア", "Ethiopia": "エチオピア", "Finland": "フィンランド", "Fiji": "フィジー", "Falkland Islands (Malvinas)": "フォークランド諸島（マルビナス）", "France": "フランス", "Faroe Islands": "フェロー諸島", "Micronesia, Federated States of": "ミクロネシア連邦", "Gabon": "ガボン", "United Kingdom": "イギリス", "Georgia": "ジョージア", "Guernsey": "ガーンジー", "Ghana": "ガーナ", "Gibraltar": "ジブラルタル", "Guinea": "ギニア", "Guadeloupe": "グアドループ", "Gambia": "ガンビア", "Guinea-Bissau": "ギニアビサウ", "Equatorial Guinea": "赤道ギニア", "Greece": "ギリシャ", "Grenada": "グレナダ", "Greenland": "グリーンランド", "Guatemala": "グアテマラ", "French Guiana": "仏領ギアナ", "Guam": "グアム", "Guyana": "ガイアナ", "Hong Kong": "香港", "Heard Island and McDonald Islands": "ハード島とマクドナルド諸島", "Honduras": "ホンジュラス", "Croatia": "クロアチア", "Haiti": "ハイチ", "Hungary": "ハンガリー", "Indonesia": "インドネシア", "Isle of Man": "マン島", "India": "インド", "British Indian Ocean Territory": "英領インド洋地域", "Ireland": "アイルランド", "Iran, Islamic Republic of": "イラン・イスラム共和国", "Iraq": "イラク", "Iceland": "アイスランド", "Israel": "イスラエル", "Italy": "イタリア", "Jamaica": "ジャマイカ", "Jersey": "ジャージー", "Jordan": "ヨルダン", "Japan": "日本", "Kazakhstan": "カザフスタン", "Kenya": "ケニア", "Kyrgyzstan": "キルギス", "Cambodia": "カンボジア", "Kiribati": "キリバス", "Saint Kitts and Nevis": "セントクリストファー・ネイビス", "Korea, Republic of": "大韓民国", "Kuwait": "クウェート", "Lao People's Democratic Republic": "ラオス人民民主共和国", "Lebanon": "レバノン", "Liberia": "リベリア", "Libya": "リビア", "Saint Lucia": "セントルシア", "Liechtenstein": "リヒテンシュタイン", "Sri Lanka": "スリランカ", "Lesotho": "レソト", "Lithuania": "リトアニア", "Luxembourg": "ルクセンブルク", "Latvia": "ラトビア", "Macao": "マカオ", "Saint Martin (French part)": "サンマルタン（フランス領）", "Morocco": "モロッコ", "Monaco": "モナコ", "Moldova, Republic of": "モルドバ共和国", "Madagascar": "マダガスカル", "Maldives": "モルディブ", "Mexico": "メキシコ", "Marshall Islands": "マーシャル諸島", "North Macedonia": "北マケドニア", "Mali": "マリ", "Malta": "マルタ", "Myanmar": "ミャンマー", "Montenegro": "モンテネグロ", "Mongolia": "モンゴル", "Northern Mariana Islands": "北マリアナ諸島", "Mozambique": "モザンビーク", "Mauritania": "モーリタニア", "Montserrat": "モントセラト", "Martinique": "マルティニーク", "Mauritius": "モーリシャス", "Malawi": "マラウイ", "Malaysia": "マレーシア", "Mayotte": "マヨット", "Namibia": "ナミビア", "New Caledonia": "ニューカレドニア", "Niger": "ニジェール", "Norfolk Island": "ノーフォーク島", "Nigeria": "ナイジェリア", "Nicaragua": "ニカラグア", "Niue": "ニウエ", "Netherlands": "オランダ", "Norway": "ノルウェー", "Nepal": "ネパール", "Nauru": "ナウル", "New Zealand": "ニュージーランド", "Oman": "オマーン", "Pakistan": "パキスタン", "Panama": "パナマ", "Pitcairn": "ピトケアン", "Peru": "ペルー", "Philippines": "フィリピン", "Palau": "パラオ", "Papua New Guinea": "パプアニューギニア", "Poland": "ポーランド", "Puerto Rico": "プエルトリコ", "Korea, Democratic People's Republic of": "朝鮮民主主義人民共和国", "Portugal": "ポルトガル", "Paraguay": "パラグアイ", "Palestine, State of": "パレスチナ", "French Polynesia": "仏領ポリネシア", "Qatar": "カタール", "Réunion": "レユニオン", "Romania": "ルーマニア", "Russian Federation": "ロシア連邦", "Rwanda": "ルワンダ", "Saudi Arabia": "サウジアラビア", "Sudan": "スーダン", "Senegal": "セネガル", "Singapore": "シンガポール", "South Georgia and the South Sandwich Islands": "サウスジョージア・サウスサンドウィッチ諸島", "Saint Helena, Ascension and Tristan da Cunha": "セントヘレナ・アセンションおよびトリスタンダクーニャ", "Svalbard and Jan Mayen": "スヴァールバル諸島およびヤンマイエン", "Solomon Islands": "ソロモン諸島", "Sierra Leone": "シエラレオネ", "El Salvador": "エルサルバドル", "San Marino": "サンマリノ", "Somalia": "ソマリア", "Saint Pierre and Miquelon": "サンピエール島・ミクロン島", "Serbia": "セルビア", "South Sudan": "南スーダン", "Sao Tome and Principe": "サントメ・プリンシペ", "Suriname": "スリナム", "Slovakia": "スロバキア", "Slovenia": "スロベニア", "Sweden": "スウェーデン", "Eswatini": "エスワティニ", "Sint Maarten (Dutch part)": "シント・マールテン（オランダ領）", "Seychelles": "セーシェル", "Syrian Arab Republic": "シリア・アラブ共和国", "Turks and Caicos Islands": "タークス・カイコス諸島", "Chad": "チャド", "Togo": "トーゴ", "Thailand": "タイ", "Tajikistan": "タジキスタン", "Tokelau": "トケラウ", "Turkmenistan": "トルクメニスタン", "Timor-Leste": "東ティモール", "Tonga": "トンガ", "Trinidad and Tobago": "トリニダード・トバゴ", "Tunisia": "チュニジア", "Türkiye": "トルコ", "Tuvalu": "ツバル", "Taiwan, Province of China": "台湾", "Tanzania, United Republic of": "タンザニア", "Uganda": "ウガンダ", "Ukraine": "ウクライナ", "United States Minor Outlying Islands": "アメリカ合衆国外諸島", "Uruguay": "ウルグアイ", "United States": "アメリカ合衆国", "Uzbekistan": "ウズベキスタン", "Holy See (Vatican City State)": "聖座（バチカン市国）", "Saint Vincent and the Grenadines": "セントビンセントおよびグレナディーン諸島", "Venezuela, Bolivarian Republic of": "ベネズエラ・ボリバル共和国", "Virgin Islands, British": "英領ヴァージン諸島", "Virgin Islands, U.S.": "米領ヴァージン諸島", "Viet Nam": "ベトナム", "Vanuatu": "バヌアツ", "Wallis and Futuna": "ウォリス・フツナ", "Samoa": "サモア", "Yemen": "イエメン", "South Africa": "南アフリカ", "Zambia": "ザンビア", "Zimbabwe": "ジンバブエ"}

In [3]:
def is_target_format(data):
    """
    入力データがすでに変換後のフォーマットに一致しているかを確認。
    """
    if isinstance(data, list):  # データがリストであることを確認
        for item in data:
            if not isinstance(item, dict) or "lat" not in item or "lng" not in item:
                return False
        return True
    return False

In [23]:

# 緯度と経度から国名を返す
def get_country_name(latitude, longitude):
    geolocator = Nominatim(user_agent="abc")
    location = geolocator.reverse((latitude, longitude), language='en')
    
    if location and 'country' in location.raw['address']:
        return country_dict[location.raw['address']['country']]
    else:
        return "unknown"

In [24]:
get_country_name(-0.6039565505133853, 34.993097157789215)


'ケニア'

#### JSON構造修正して国名のタグ追加

In [25]:
# 入力ファイルと出力ファイルのパス
input_file = "./input/map-making-app_1.json"
output_file = "./output/geoguessr_4.json"

# JSONファイルを読み込んで、customCoordinatesを抽出し、必要なら変換後フォーマットに変更
try:
    with open(input_file, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    if is_target_format(data):
        # 既に変換後のフォーマットの場合
        formatted_data = data
        print("入力データは既に変換フォーマットです。")
    else:
        # customCoordinatesを抽出
        custom_coordinates = data.get("customCoordinates", None)

        if custom_coordinates is None:
            print("customCoordinatesが見つかりませんでした。")
            formatted_data = []
        else:
            # customCoordinatesを出力データに設定
            formatted_data = custom_coordinates
            print("customCoordinatesを抽出しました。")

    cnt = 1
    # 各要素にextra属性を追加
    for item in formatted_data:
        if isinstance(item, dict) and "lat" in item and "lng" in item:
            country_name = get_country_name(float(item["lat"]), float(item["lng"]))

            if "extra" in item:
                # extraが既に存在する場合、tagsに"add"を追加
                if "tags" in item["extra"] and isinstance(item["extra"]["tags"], list):
                    item["extra"]["tags"].append(country_name)
                else:
                    # tagsが存在しない場合、新規作成
                    item["extra"]["tags"] = [country_name]
            else:
                # extraが存在しない場合、新規作成
                item["extra"] = {
                    "tags": [country_name],
                    "panoId": None,
                    "panoDate": None,
                }
        # 進捗を表示
        print(cnt , "/", len(formatted_data))
        cnt += 1

    # 結果を新しいJSONファイルに書き込む
    with open(output_file, "w", encoding="utf-8") as outfile:
        json.dump(formatted_data, outfile, ensure_ascii=False, indent=4)
    print(f"結果が {output_file} に保存されました。")

except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")

customCoordinatesを抽出しました。
1 / 4
2 / 4
3 / 4
4 / 4
結果が ./output/geoguessr_4.json に保存されました。


#### JSON形式だけ変換（タグ情報追加せず）

In [3]:
# 入力ファイルと出力ファイルのパス
input_file = './input/map-making-app_1.json'
output_file = './output/geoguessr_3.json'

# JSONファイルを読み込んで、customCoordinatesを抽出し、必要なら変換後フォーマットに変更
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
    
    if is_target_format(data):
        # 既に変換後のフォーマットの場合
        output_data = data
        print("入力データは既に変換フォーマットです。")
    else:
        # customCoordinatesを抽出
        custom_coordinates = data.get('customCoordinates', None)
        
        if custom_coordinates is None:
            print("customCoordinatesが見つかりませんでした。")
            output_data = []
        else:
            # customCoordinatesを出力データに設定
            output_data = custom_coordinates
            print("customCoordinatesを抽出しました。")
    
    # 結果を新しいJSONファイルに書き込む
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(output_data, outfile, ensure_ascii=False, indent=4)
    print(f"結果が {output_file} に保存されました。")

except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")


customCoordinatesを抽出しました。
結果が ./output/geoguessr_3.json に保存されました。
